# NOTEBOOK FOR DATASET CREATION

THIS NOTEBOOK IS FOR DATASET CREATION , ANYONE CAN MAKE THEIR OWN DATASET USING THIS NOTEBOOK

In [ ]:
#IMPORTING NECESSARY LIBRARIES
import numpy as np
import cv2
import os
import time

This function is used to accumulate weight of background , so that we can detect if any object coming into the frame ,This method basically used for object detection 
I am providing is an small article from where you can get intuition
LINK: https://opencvpython.blogspot.com/2012/07/background-extraction-using-running.html

In [2]:
background=None
alpha=0.5
def accumulated_weight(frame,alpha):
    global background
    if(background is None):
        background=frame.copy().astype("float")
    cv2.accumulateWeighted(frame,background,alpha)
    

Hand Segmentation is used to detect hand by calculating difference between foreground and background when hand comes into the frame

In [3]:
def hand_segmentation(frame,threshold=25):
    global background
    diff=cv2.absdiff(background.astype('uint8'),frame)
    
    _,thresholded=cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY) 
    
    '''Used for thresholding the image(i.e If a pixel value is greater than threshold provided, pixel value 255 is given and 
    0 if pixel value is smaller than threshohld)'''
    
    contours,hei=cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    '''findContours is used to find boundaries in a image for example finding  boundary of a rectangle in a image in this case 
    it will find boundaries of our hand and it prefers to work with thresholded images'''
    
    if(len(contours)==0):
        return(None)
    
    else:
        hand_segment_max=max(contours,key=cv2.contourArea)
        return(thresholded,hand_segment_max)
    

In [4]:
dic={1:"A",2:"B",3:"C",4:"D",5:"E",6:"F",7:"G",8:"H",9:"I",10:"J",11:"K",12:"L",13:"M",14:"N",
          15:"O",16:"P",17:"Q",18:"R",19:"S",20:"T",21:"U",22:"V",23:"W",24:"X",25:"Y",26:"Z"}


for i in range(1,len(dic)+1): #Running loop for all 26 alphabets
    time.sleep(5) #Pausing the loop for 5 seconds to adjust for  next ASL signs
    print("ADJUST ANOTHER SIGN IN FIVE SECONDS")
    
    cam=cv2.VideoCapture(0) #Opening the camera
    num_frames=0
    element=i
    num_images_taken=0
    
    #Frame where we adjust our hand and create dataset
    ROI_top = 100
    ROI_bottom = 300
    ROI_right = 150
    ROI_left = 350

    while(True):
        ret,frame=cam.read() #reading real time camera pictures
    
        frame=cv2.flip(frame,1) #flipping the camera 
    
        frame_copy=frame.copy() #making copy of a frame (Real time display i.e the bigger one)
    
        roi=frame[ROI_top:ROI_bottom,ROI_right:ROI_left] #Region of Interest
    
        gray_frame=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY) #Converting Frame to RGB to gray
    
        gray_frame=cv2.GaussianBlur(gray_frame,(9,9),0) #Gaussian Blurring(Refer GFG)
    
        if(num_frames<60): '''This loop is for fetching the background 
        so that it can calculate difference between foreground and background (60 frames will be enough)'''
            
            accumulated_weight(gray_frame,alpha)
            if(num_frames<=59):
            
                cv2.putText(frame_copy,"FETCHING BACKGROUND...PLEASE WAIT",(80,400),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,0,255),2)
            
        elif(num_frames<=300): 
            '''Adjusting hand gestures so that dataset will be good created(It is important) '''
            hand=hand_segmentation(gray_frame)
        
            cv2.putText(frame_copy,"ADJUST HAND FOR"+ dic[element],(200,400),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
        
            if(hand is not None): #If hand is detected in the frame 
                thresholded,hand_segment=hand
            
                cv2.drawContours(frame_copy,[hand_segment+(ROI_right,ROI_top)],-1,(255,0,0),2) #Drawing Contours(Blue Color)
            
                cv2.putText(frame_copy, str(num_frames)+"For" + dic[element],(70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
                cv2.imshow("THRESHOLDED IMAGE",thresholded) #Displaying the thresholded Image
            
            
        else:
            '''This loop is for saving image into local memory (Upto 300 images for each alphabet)'''
            hand=hand_segmentation(gray_frame)
        
            if(hand is not None):
                thresholded,hand_segment=hand
            
                cv2.drawContours(frame_copy, [hand_segment + (ROI_right,ROI_top)], -1, (255, 0, 0),1)
            
                cv2.putText(frame_copy, str(num_frames)+"For" + dic[element],(70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
                cv2.putText(frame_copy, str(num_images_taken) + 'images' +"For"+ dic[element],(200, 400), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255), 2)
            
                cv2.imshow("THRESHOLDED IMAGE",thresholded)
            
                if num_images_taken <= 300:
                    directory=r"C:/Users/Aryan Chauhan/Desktop/AIO/GESTURE/TRAIN" #Directory where we want to save our data
                    os.chdir(directory) #changing the directory
                    directory_for_elements=dic[element]  
                    path=os.path.join(directory,directory_for_elements) #Making directory for each alphabet
                    try:
                        os.mkdir(directory_for_elements) #Making of new directory
                    except OSError as error:
                        pass
                    os.chdir(directory_for_elements) 
                    filename=dic[element]+str(num_images_taken+300)+".jpg"
                    h=cv2.imwrite(filename, thresholded)  #Saving the images
                    
                
                else:
                    break
                
                num_images_taken+=1
            
            else:
                cv2.putText(frame_copy,"NO HAND DETECTED",(200,400),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
            
            
        cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,ROI_bottom), (255,128,0), 3)
    
        cv2.putText(frame_copy, "Hand sign recognition_ _ _", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    
        num_frames+=1
    
        cv2.imshow("SIGN DETECTION",frame_copy)
    
        if(cv2.waitKey(25) & 0xFF==ord('q')):
            cv2.destroyAllWindows()
            break

cv2.destroyAllWindows()
cam.release()
        


ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
ADJUST ANOTHER SIGN IN FIVE SECONDS
